# NDVI Change Detection


- **Special requirements:** A Google account, access to Google Earth Engine.

- **Prerequisites:** You should have completed the "Week 4 - Prac 1" and "Week 5 - Prac 1" notebooks.

## Description
Many earth observation datasets are available at regular intervals over long periods of time. This enables us to detect changes on the Earth’s surface. In this session, you will learn how to apply change detection technique in remote sensing used to create NDVI change detection maps. 

## Aims of the practical session
* Create ROI and load it
* Collect images 
* Calculate NDVI for before and after Image
* Comparing two classified images to see the changes of NDVI during the time

## Getting started

### Load packages

Import GEE packages that are needed for the analysis.

In [1]:
import ee
import geemap
# ee.Authenticate()

### Connect to Google Earth Engine (GEE)

Connect to the GEE to have access computing tools and GEE datasets.
You may be required to input your Google account for authorization.

In [2]:
Map = geemap.Map()
# Map.add_basemap('HYBRID')
Map.addLayerControl()

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Adding Region of Interest (ROI)

Create ROI that we want to work on it and then add and display it on the GEE map.
Import the downloaded shapefile for Canberra central from your computer path as ROI.

In [3]:
# # # load shapefile (ROI)
shp_path = 'C:/UserData/abdollaa/OneDrive - Australian National University/Tutorials/GEE/NOV21_ACT_LOC_POLYGON_shp/Canberra_central.shp'
geometry = geemap.shp_to_ee(shp_path)
Map.addLayer(geometry, {}, 'Canberra central SHP')
Map.centerObject(geometry);

### Post Image collection 
An ImageCollection is a stack or sequence of images. An ImageCollection can be loaded by pasting an Earth Engine asset ID into the ImageCollection constructor. You can find ImageCollection IDs in the <a href="https://developers.google.com/earth-engine/datasets">data catalog</a>. 

We will:
* Load post Sentinel-2 images for the anlysis
* Filter a collection by date range
* Select bands
* Clip based on geometry
* Visualize Image

In [4]:
# def maskS2clouds(image):
#     qa = image.select('QA60')
#     # Bits 10 and 11 are clouds and cirrus, respectively.
#     cloudBitMask = 1 << 10
#     cirrusBitMask = 1 << 11
#     # Both flags should be set to zero, indicating clear conditions.
#     mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
#         .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
#     return image.updateMask(mask).divide(10000) #re-scale 

In [5]:
## collect before event images 
imagePo = (
    ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(geometry)
    .filterDate('2021-01-01', '2021-12-31') ### Note: you can try different dates
    #.map(maskS2clouds) #map the cloudmasking/rescaling function
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  #filter the collection by the CLOUD_COVER property
    .select('B.*')
)

In [6]:
print(imagePo.size().getInfo());

49


In [7]:
# # # Visualize Image
vis_params = {'min': 0, 'max':  3000, 'bands': ['B4', 'B3', 'B2']}

Map.addLayer(imagePo.median().clip(geometry), vis_params, 'Sent-2 After Image' );
Map

Map(center=[-35.28672794816286, 149.1314839394662], controls=(WidgetControl(options=['position', 'transparent_…

### Calculate NDVI time-series for post image
The purpose of this part is to calculate time series NDVI on remotely sensed data. The NDVI is a vegetation index that describes the difference between visible and near-infrared reflectance of vegetation cover and can be used to estimate the density of green on an area of land.

We will:
* Define a function to calculate NDVI
* Perform time series of the study area based on post imagery
* Clip based on geometry
* Map the results

### Create post-NDVI map

In [8]:
#Create Post-NDVI Map

def func_ndvipo(img):
  index = img.normalizedDifference(['B8', 'B4'])
  clip = index.clip(geometry);
  return clip.rename('Ndvi').copyProperties(img,['system:time_start','system:time_end'])

ndviPo = imagePo.map(func_ndvipo);

In [9]:
# mean value composite for Before image(calculate mean value of ndvi in every pixel)
mvcPo = ndviPo.mean();

In [10]:
Map.addLayer(mvcPo, {min: -1, max: 1, 'palette': ['white', 'red']}, 'NDVI After Event');
Map

Map(center=[-35.28672794816286, 149.1314839394662], controls=(WidgetControl(options=['position', 'transparent_…

In [11]:
#print(mvcPo.getInfo())

### Create chart for post image
We can use ipygee to generate a chart of the NDVI variation in the analysis period for any point in the ROI:

In [12]:
from ipygee import chart

ndviPo = chart.Image.series(**{
    'imageCollection': ndviPo,
    'region': geometry,
    'reducer': ee.Reducer.mean(),
    'scale': 20,
    'xProperty': 'system:time_start'})
ndviPo.renderWidget(width='100%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

### Pre Image collection 

We will:
* Load pre Sentinel-2 images for the anlysis
* Filter a collection by date range
* Select bands
* Clip based on geometry
* Visualize Image

In [13]:
#Import Pre-Event Imagery
imagePre = (
    ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(geometry)
    .filterDate('2017-01-01', '2017-12-31') ### Note: you can try different dates
    #.map(maskS2clouds) #map the cloudmasking/rescaling function
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .select('B.*')
)

print(imagePre.size().getInfo())

34


In [14]:
# # # Visualize Image
vis_params = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

Map.addLayer(imagePre.median().clip(geometry), vis_params, 'Sent-2 Before Image' );
Map

Map(center=[-35.28672794816286, 149.1314839394662], controls=(WidgetControl(options=['position', 'transparent_…

### Calculate NDVI time-series for pre image

We will:
* Define a function to calculate NDVI
* Perform time series of the study area based on pre imagery
* Clip based on geometry
* Map the results

In [15]:
#Create pre-NDVI Map

def func_ndvipre(img):
  index = img.normalizedDifference(['B8', 'B4'])
  clip = index.clip(geometry);#for more regions is not necessary
  return clip.rename('Ndvi').copyProperties(img,['system:time_start','system:time_end'])

ndviPre = imagePre.map(func_ndvipre)

In [16]:
# mean value composite for Before image(calculate mean value of ndvi in every pixel)
mvcPre = ndviPre.mean();

In [17]:
Map.addLayer(mvcPre, {min: -1, max: 1, 'palette': ['white', 'red']}, 'NDVI Before Event');
Map

Map(center=[-35.28672794816286, 149.1314839394662], controls=(WidgetControl(options=['position', 'transparent_…

### Create chart for pre image
We can use ipygee to generate a chart of the NDVI variation in the analysis period for any point in the ROI:

In [18]:
from ipygee import chart

ndviPre = chart.Image.series(**{
    'imageCollection': ndviPre,
    'region': geometry,
    'reducer': ee.Reducer.mean(),
    'scale': 20,
    'xProperty': 'system:time_start'})
ndviPre.renderWidget(width='100%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

### <a name="ex3"></a> Challenge

<div class="alert alert-block alert-danger">

- Try to improve the NDVI plots by setting options like changing xlabel/ylabel, select good title, change font/font size etc.
    
    
</div>    

### Create NDVI Difference Map
Here, we apply codes to create NDVI difference map based on pre and post results.
We will: 
* Create NDVI Difference Map
* show differences
* Map diff results

In [19]:
# # # Create NDVI Difference Map
diff = mvcPo.subtract(mvcPre);
#print(diff.getInfo())

In [20]:
Map.addLayer(diff,
            {min: -2, max: 2, 'palette': ['white', 'red']},
            'NDVI Difference');
Map

Map(center=[-35.28672794816286, 149.1314839394662], controls=(WidgetControl(options=['position', 'transparent_…

### Export the result
Export the result directly to your computer

In [21]:
# # # Export the result
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'NDVI_Diff.tif')

In [22]:
diffClip = diff.clipToBoundsAndScale(**{
  'geometry': geometry,
  'scale': 20,  # meters
})

In [24]:
geemap.ee_export_image(diffClip, filename=out_file)

<div class="alert alert-block alert-danger"> 

### Exercise 1 - NDVI change detection
Try to analyze trend change detection in NDVI time series and then try to check the following exercises:
* What wavelengths are used in NDVI?
* How do you analyze NDVI data?
* Is the results of NDVI varied in two different times? Why?
* What are the factors affecting NDVI value with respect to time change?
<div>

## References
This is where the references go. For exmaple:

* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305
* "Earth Observation: Data, Processing and Applications" book. Available through Wattle, or http://www.crcsi.com.au/earth-observation-series.

## Additional information

**License:** The code in this notebook was initially created by the team at [Digital Earth Australia](https://github.com/GeoscienceAustralia/dea-notebooks), and has been modified by Abolfazl Abdollahi. The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle site.

**Last modified:** August 2022